In [5]:
import pandas as pd
import numpy as np
from numba import njit

# Helper functions

In [6]:
@njit()
def prob_func(A,E,likelihood):
    
    """
        A - active banks (type:float)
        E - number of undirected edges (type:float)
        likelihood - likelihood function given a kappa and Np,
        (type: B x 3 numpy array, where B is the number combinations of N,M generated from 
        kappa and Np)
        Returns the probability of observing A and E given a kappa and Np.
    """
    
    probindex = np.where((likelihood[0:,0]==A) & (likelihood[0:,1]==E))
    if probindex[0].size!=0:
        probability = likelihood[probindex][0,-1]
    
    return probability

In [7]:
def estimators(estimates, params):
    """
    Input. 
    1. estimates: Sequences of IDs for (kappa, Np) combinations which have been
    identified as estimators by the method.
    2. params: Sequence of parameter combinations. Dict type i.e., parameter id: (kappa, Np)
    
    Output: DataFrame of estimates kappa_hat and Np_hat given a time window t
    
    The function matches the ids of the parameter combination with the ML
    estimators (kappa_hat and Np_hat).
    """
    kappa_seq = []
    np_seq = []
    for combs in estimates["params"]:
        kappa,np = params[combs]
        kappa_seq.append(kappa)
        np_seq.append(np)
    return kappa_seq,np_seq

# Maximum-likelihood estimation

In [8]:
def MLEstimation(NMseq,likelihood_funcs,params):
    """
    Input.
    
    1.NMseq: Sequence of triplets consisting of time (t), total active nodes (N) and total edges (M).
    2.likelihood_funcs: Sequence of joint prob. dists. Dict type i.e., parameter combination id: joint prob dist
    3.params: Sequence of parameter combinations. Dict type i.e., parameter id: (kappa, Np)
    
    Output. 
    estimates: Estimates of overall activity and population size. Type: (pandas.DataFrame)
    """
    knp_ests = []
    timeseq = []
    for elem in NMseq:
        t,N,M = elem
        prob_NM = []
        combid = []
        for fid,LF in likelihood_funcs.items():
            prob_NM.append(prob_func(N,M,LF))
            combid.append(fid)
        estparam_index = np.argmax(np.asarray(prob_NM))
        estparams_id = combid[estparam_index]
        knp_ests.append( estparams_id)
        timeseq.append(t)
        MLEresults = dict(zip(timeseq,knp_ests))
    #
    estimates = pd.DataFrame.from_dict(MLEresults,orient = "index")
    estimates.reset_index(drop = False, inplace = True)
    estimates.rename(columns = {"index":"t",0:"params"}, inplace = True)
    kappavalues, npvalues = estimators(estimates,params)
    estimates["kappa"] = kappavalues
    estimates["np"] = npvalues
    return estimates